In [ ]:
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import plotly.express as px


In [ ]:
df=pd.read_csv('https://raw.githubusercontent.com/hohohohia/misc/main/full_ds.csv')
df.head()

,Unnamed: 0,MapDate,FIPS,County,State,None,D0,D1,D2,D3,...,StatisticFormatID,wsum,FIPS STR,roff,evap,smap,PRCP,SNWD,TMAX,TMIN
0,1593,2011-12-27,1007,Bibb County,AL,0.00,100.00,54.54,0.0,0.0,...,1,209.08,1007,-9999.0,-9999.0,-9999.0,0.000000,NaN,164.000000,55.500000
1,1594,2012-01-03,1007,Bibb County,AL,0.00,100.00,54.54,0.0,0.0,...,1,209.08,1007,-9999.0,-9999.0,-9999.0,48.714286,NaN,170.000000,40.571429
2,1595,2012-01-10,1007,Bibb County,AL,0.00,100.00,54.54,0.0,0.0,...,1,209.08,1007,-9999.0,-9999.0,-9999.0,33.714286,NaN,148.428571,25.428571
3,1596,2012-01-17,1007,Bibb County,AL,41.41,58.59,0.00,0.0,0.0,...,1,58.59,1007,-9999.0,-9999.0,-9999.0,102.285714,NaN,196.142857,95.428571
4,1597,2012-01-24,1007,Bibb County,AL,100.00,0.00,0.00,0.0,0.0,...,1,0.00,1007,-9999.0,-9999.0,-9999.0,199.571429,NaN,193.571429,27.000000


In [ ]:
df.columns

Index(['Unnamed: 0', 'MapDate', 'FIPS', 'County', 'State', 'None', 'D0', 'D1',
       'D2', 'D3', 'D4', 'ValidStart', 'ValidEnd', 'StatisticFormatID', 'wsum',
       'FIPS STR', 'roff', 'evap', 'smap', 'PRCP', 'SNWD', 'TMAX', 'TMIN'],
      dtype='object')

In [ ]:
Y=np.array(df.to_numpy()[:,[5,6,7,8,9,10]],dtype=float)
Yt=np.transpose(Y)
Y

array([[  0.  , 100.  ,  54.54,   0.  ,   0.  ,   0.  ],
       [  0.  , 100.  ,  54.54,   0.  ,   0.  ,   0.  ],
       [  0.  , 100.  ,  54.54,   0.  ,   0.  ,   0.  ],
       ...,
       [  0.  , 100.  , 100.  , 100.  ,  16.75,   0.  ],
       [  0.  , 100.  , 100.  , 100.  ,  52.39,   0.  ],
       [  0.  , 100.  , 100.  , 100.  ,  52.39,   0.  ]])

In [ ]:
Y_tsne = TSNE(n_components=3, learning_rate='auto', init='random',n_iter=5000,n_iter_without_progress=1000).fit_transform(Yt)
Y_pca = PCA(n_components=3).fit_transform(Yt)

In [ ]:
groups = pd.DataFrame(Y_tsne,columns=['x', 'y','z']).assign(category=['None','D0','D1','D2','D3','D4'])
fig = px.scatter_3d(groups, x='x', y='y', z='z',color='category')
fig.show()


In [ ]:
groups = pd.DataFrame(Y_pca,columns=['x', 'y','z']).assign(category=['None','D0','D1','D2','D3','D4'])
fig = px.scatter_3d(groups, x='x', y='y', z='z',color='category')
fig.show()


In [ ]:
# percent of none>threshold
len(np.where(Y[:,0]=100)[0])/len(Y[:,0])

0.5489181837204437

49% county-time slice (row) has None>90, meaning that 90% area of that county-time slice is not under the impact of any level of drought. We define this as `NO DROUGHT`, and the rest is `DROUGHT`. Thus, we model the drought prediction task in a binary way.

In [ ]:
srdf=pd.read_csv('https://raw.githubusercontent.com/hohohohia/misc/main/semi_reduced_ds.csv')
srdf

,MapDate,FIPS,County,State,None,D0,D1,D2,D3,D4,roff,evap,smap
0,12/30/2014,1007,Bibb County,AL,100.0,0.0,0.0,0.0,0.0,0.0,0.001730,-3.701070e-04,-9999.000000
1,1/6/2015,1007,Bibb County,AL,100.0,0.0,0.0,0.0,0.0,0.0,0.000068,-9.940000e-05,-9999.000000
2,1/13/2015,1007,Bibb County,AL,100.0,0.0,0.0,0.0,0.0,0.0,0.000077,-1.348450e-04,-9999.000000
3,1/20/2015,1007,Bibb County,AL,100.0,0.0,0.0,0.0,0.0,0.0,0.000395,-3.874420e-04,-9999.000000
4,1/27/2015,1007,Bibb County,AL,100.0,0.0,0.0,0.0,0.0,0.0,0.000193,-2.163510e-04,-9999.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
163963,11/30/2021,56043,Washakie County,WY,0.0,100.0,100.0,100.0,0.0,0.0,0.000002,-2.080000e-05,0.180940
163964,12/7/2021,56043,Washakie County,WY,0.0,100.0,100.0,100.0,0.0,0.0,0.000002,-8.490000e-06,0.186998
163965,12/14/2021,56043,Washakie County,WY,0.0,100.0,100.0,100.0,0.0,0.0,0.000002,-5.070000e-05,0.187852
163966,12/21/2021,56043,Washakie County,WY,0.0,100.0,100.0,100.0,0.0,0.0,0.000002,-2.680000e-05,0.187990


In [ ]:
D=np.where(srdf['None'].to_numpy()>90,1,0)
np.sum(D)/len(D)


0.537903737314598

In [ ]:
D=np.array(D,dtype=str)

This binary cut is stable! see on our semi reduced dataset, the percentage of None>90 is still around 50%.

let's cluster features such as runoff, evaporation, soil moisture. See if they have a clear seperation intrinsicly. 

In [ ]:
features=np.array(srdf.to_numpy()[:,[10,11,12]],dtype=float)

features_pca = PCA(n_components=3).fit_transform(features)
groups = pd.DataFrame(features_pca,columns=['x', 'y','z']).assign(category=D)


In [ ]:
fig = px.scatter_3d(groups, x='x', y='y',z='z',color='category')
fig.show()

Output hidden; open in https://colab.research.google.com to view.

The 2 classes have no clear seperation. This reamains a very challenging and meaningful ML task to be learned. 

In [ ]:
features_tsne = TSNE(n_components=3).fit_transform(features)
groups = pd.DataFrame(features_tsne,columns=['x', 'y','z']).assign(category=D)


/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



In [ ]:
fig = px.scatter_3d(groups, x='x', y='y',z='z',color='category')
fig.show()